## langchain
Some examples/messing around with langchain. There's three main headers, run the initial pip3 install/import located below this and then run code under headers in order.  
Ensure you fill in the OPENAI_API_KEY variable in the cells below. It's used by the OpenAI() function. Environment variable makes it easier.

In [ ]:
!pip3 install langchain openai beautifulsoup4 tiktoken faiss-cpu chromadb

In [24]:
import os
from langchain.document_loaders import ReadTheDocsLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI

# Set the environment variable
os.environ['OPENAI_API_KEY'] = "Your key here, get from here, they give you an initial amount of credit for free https://platform.openai.com/account/api-keys"

## Just OpenAI
Loads the a chat object that allows users to send messages and receive messages from the API. Context is saved per message allowing the LLM to use previous messages as context. Giving history when using API.

In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

messages = []

i = 0
while i < 3:
    message = input("> ")
    usr_msg = HumanMessage(content=message)
    messages.append(usr_msg)
    ai_msg = chat(messages)
    print(ai_msg.content)
    messages.append(ai_msg)
    i += 1

>  dsa


I'm sorry, I don't understand what you mean by "dsa". Can you please provide more context or information?


>  d


I'm sorry, I still don't understand what you mean by "d". Can you please provide more context or information?


>  What did I just write?


You wrote "d".


## Read the Docs
Download RtD site with wget. Save to folder. Then use the RtD loader, parsing said information, splitting text, using chroma for vector storage, and then running Q/A against it.

In [ ]:
!wget -r -A.html -P rtdocs https://tapis.readthedocs.io/en/latest/

In [6]:
loader = ReadTheDocsLoader("rtdocs", features='html.parser', errors='ignore')

documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000, chunk_overlap=0, separators=[" ", ",", "\n"]
    )

texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)

qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=docsearch.as_retriever())

In [8]:
#query = "Can you give me a code example of using tapipy and creating an abaco actor?"
query = "Can you give me a code example of using tapipy and creating a pod for neo4j?"
#query = "Can you give me a code example of using tapipy and creating a tapis system and uploading a file to it?"

res = qa.run(query)

print(res)

 Yes. You can use the following code example to create a pod for neo4j using tapipy. It is important to note that the tapipy library should be version 1.2.3 or higher to use the pods service:

# Import the Tapis object
from tapipy.tapis import Tapis
# Log into you the Tapis service by providing user/pass and url.
t = Tapis(base_url='https://tacc.tapis.io',
          username='your username',
          password='your password')
# Get tokens that will be used for authenticated function calls
t.get_tokens()

# To register an pod using the tapipy library, we use the pods.create_pod() method and pass the arguments describing
# the pod we want to register through the function parameters. For example:
t.pods.create_pod(pod_id='docpod', pod_template='neo4j', description='My example pod!')


# OpenAPI with Planners
Downloading the pgrest openapi schema and using an openapi agent to plan a workflow of requests based on a user query.  
It would be great to have multiple schemas readable at once, don't quite know how to do that yet.

In [10]:
!wget https://raw.githubusercontent.com/tapis-project/tapipy/main/tapipy/resources/openapi_v3-pgrest.yml

--2023-05-25 11:44:47--  https://raw.githubusercontent.com/tapis-project/tapipy/main/tapipy/resources/openapi_v3-pgrest.yml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33814 (33K) [text/plain]
Saving to: ‘openapi_v3-pgrest.yml.1’

openapi_v3-pgrest.y 100%[===================>]  33.02K  --.-KB/s    in 0.004s  

2023-05-25 11:44:48 (7.41 MB/s) - ‘openapi_v3-pgrest.yml.1’ saved [33814/33814]



In [18]:
import os, yaml
from langchain.agents.agent_toolkits.openapi.spec import reduce_openapi_spec
from langchain.llms.openai import OpenAI
from langchain.agents.agent_toolkits.openapi import planner
from langchain.requests import RequestsWrapper

with open("openapi_v3-pgrest.yml") as f:
    raw_openai_api_spec = yaml.load(f, Loader=yaml.Loader)
openai_api_spec = reduce_openapi_spec(raw_openai_api_spec)

#This is where you can set headers if you want
headers = {}
requests_wrapper = RequestsWrapper(headers=headers)

In [ ]:
llm = OpenAI(temperature=0.0)
pgrest_agent = planner.create_openapi_agent(openai_api_spec, requests_wrapper, llm)

user_query = "Use pgrest to create a new role named MYROLE, create a new table, and add a row to the table, and create a view of the table restricted to MYROLE. Don't actually run the requests, only plan"
pgrest_agent.run(user_query)

## Combining multiple toolkits/vectorstores
I'd like to be able to combine multiple toolkits (more than one create_openapi_agent for example). Logically I doubt this makes sense, but there's an issue.
- https://github.com/hwchase17/langchain/issues/3435

## Docs
Just some useful doc links for additional context. Docs have a decent amount of examples that prove useful.
- https://python.langchain.com/en/latest/reference/modules/agents.html?highlight=create_openapi_agent#langchain.agents.create_openapi_agent
- https://python.langchain.com/en/latest/modules/chains/index_examples/question_answering.html#prepare-data


In [79]:
# Mainly stolen - https://github.com/hwchase17/chat-langchain
# This is using FAISS instead of chroma for the vectorstore. Couldn't quite figure out how to use said vectorstore though. 
from langchain.document_loaders import ReadTheDocsLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma


loader = ReadTheDocsLoader("rtdoc", features='html.parser')
raw_documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
documents = text_splitter.split_documents(raw_documents)
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(documents, embeddings)

# Save vectorstore
with open("vectorstore.pkl", "wb") as f:
    pickle.dump(vectorstore, f)